### 여러 페이지에서 가져온 배우 정보를 MongoDB 에 저장하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [2]:
'''
MongoDB 연결하기
'''

conn = pymongo.MongoClient()
cine = conn.cine
collection_cine2 = cine.collection_cine2


In [3]:
'''
collection 삭제
'''
# collection_cine.drop()

'\ncollection 삭제\n'

In [4]:
actors_info_list = []

In [5]:
url = "http://www.cine21.com/rank/person/content"

post_data = {}
post_data["section"] = "actor"
post_data["period_start"] = "2020-12"
post_data["gender"] = "all"


In [6]:
# 1 page ~ 20 page


for idx in range(1, 21):

    post_data["page"] = idx

    res = requests.post(url, data=post_data)
    soup = BeautifulSoup(res.content, "html.parser")


    actors = soup.select("li.people_li div.name")
    hits   = soup.select("ul.num_info > li > strong")
    movies = soup.select("ul.mov_list")
    rankings = soup.select("li.people_li > span.grade")

    for idx, actor in enumerate(actors):
        actor_name   = (actor.text).split("(")[0]
        actor_hits   = int(hits[idx].text.replace(",",""))
        movie_titles = movies[idx].select("li a span")

        movie_title_list = []
        for movie_title in movie_titles:            
            movie_title_list.append(movie_title.text)

        actors_info_dict = { }
        actors_info_dict["배우이름"] = actor_name
        actors_info_dict["흥행지수"] = actor_hits
        actors_info_dict["출연영화"] = movie_title_list
        actors_info_dict["랭킹"] = rankings[idx].text


        actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
        actor_resp = requests.get(actor_link)    

        actor_soup = BeautifulSoup(actor_resp.content, "html.parser")

        default_info  = actor_soup.select_one("ul.default_info")
        actor_details = default_info.select("li")


        for actor_item in actor_details:           
            actor_item_key = actor_item.select_one("span.tit").text
            actor_item_value = re.sub("<span.*?>.*?</span>", "", str(actor_item))
            actor_item_value = re.sub("<.*?>", "", actor_item_value)

            actors_info_dict[actor_item_key] = actor_item_value

        actors_info_list.append(actors_info_dict)    

    
actors_info_list  


[{'배우이름': '박해일',
  '흥행지수': 26750,
  '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
  '랭킹': '1',
  '직업': '배우',
  '생년월일': '1977-01-26',
  '성별': '남',
  '신장/체중': '177cm, 65kg',
  '학교': '남서울대학교 영어학 (중퇴)',
  '취미': '영화감상, 음악감상, 컴퓨터게임',
  '특기': '노래, 기타연주, 탁구, 야구',
  '소속사': '휴메인엔터테인먼트'},
 {'배우이름': '박지환',
  '흥행지수': 24672,
  '출연영화': ['유체이탈자',
   '한산: 용의 출현',
   '해적: 도깨비 깃발',
   '지푸라기라도 잡고 싶은 짐승들',
   '범죄도시2',
   '봉오동 전투'],
  '랭킹': '2',
  '직업': '배우',
  '생년월일': '1980-09-05',
  '성별': '남'},
 {'배우이름': '손석구',
  '흥행지수': 23947,
  '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
  '랭킹': '3',
  '직업': '배우',
  '생년월일': '1983-00-00',
  '성별': '남'},
 {'배우이름': '현빈',
  '흥행지수': 21073,
  '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'],
  '랭킹': '4',
  '원어명': '玄彬',
  '다른 이름': '김태평',
  '직업': '배우',
  '생년월일': '1982-09-25',
  '성별': '남',
  '홈페이지': '\nhttp://www.vastenm.com/\n',
  '신장/체중': '184cm, 74kg',
  '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중',
  '취미': '수영, 농구, 야구',
  '특기': '스노우보드, 사진',
  '소속사': 'VAST엔터테인먼트'},
 {'배우이름': '최귀화'

In [7]:
# collection_cine.delete_many({})

In [8]:
collection_cine2.insert_many(actors_info_list)

In [9]:
collection_cine2.count_documents({})

140

### 데이터 분석 준비하기
#### 1) key(field)명 변경하기
- MongoDB 에 저장되어 있는 document 의 key(field) 이름 변경하기
- update_one() / update_many()

In [10]:
collection_cine2.find_one({})

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '배우이름': '박해일',
 '흥행지수': 26750,
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '랭킹': '1',
 '직업': '배우',
 '생년월일': '1977-01-26',
 '성별': '남',
 '신장/체중': '177cm, 65kg',
 '학교': '남서울대학교 영어학 (중퇴)',
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '소속사': '휴메인엔터테인먼트'}

In [11]:
import pprint

documents = collection_cine2.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3893'),
 '랭킹': '2',
 '배우이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3894'),
 '랭킹': '3',
 '배우이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}
--------------------------------------------------------------------

In [12]:
'''
배우이름 --> 배우 이름  : $rename
'''
collection_cine2.update_many({}, {"$rename" : {"배우이름":"배우 이름"}})

In [13]:
documents = collection_cine2.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3893'),
 '랭킹': '2',
 '배우 이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3894'),
 '랭킹': '3',
 '배우 이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}
-----------------------------------------------------------------

### 조회할 때 정렬하기(오름차순 / 내림차순)
- sort() 메소드
- 오름차순 : pymongo.ASCENDING (기본값)
- 내림차순 : pymongo.DESCENDING

In [14]:
documents = collection_cine2.find({}).sort("생년월일").limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c38a1'),
 '랭킹': '16',
 '배우 이름': '이정재',
 '성별': '남',
 '소속사': '아티스트컴퍼니',
 '직업': '배우',
 '출연영화': ['헌트', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '젊은 남자'],
 '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n',
 '흥행지수': 15122}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38a9'),
 '랭킹': '24',
 '배우 이름': '김성규',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '악인전'],
 '홈페이지': '\nhttps://www.instagram.com/kimsungkyu_official/\n',
 '흥행지수': 11612}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38ae'),
 '랭킹': '29',
 '배우 이름': '김소진',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년'],
 '흥행지수': 10280}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38b0'),
 '랭킹': '31',
 '배우 이름': '이규원',
 '성별': '남',
 '직

In [15]:
documents = collection_cine2.find({}).sort("생년월일", pymongo.ASCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c38a1'),
 '랭킹': '16',
 '배우 이름': '이정재',
 '성별': '남',
 '소속사': '아티스트컴퍼니',
 '직업': '배우',
 '출연영화': ['헌트', '신과 함께-죄와 벌', '태양은 없다', '시월애', '신과 함께-인과 연', '젊은 남자'],
 '홈페이지': '\nhttp://www.artistcompany.co.kr/portfolio-item/lee-jung-jae/\n',
 '흥행지수': 15122}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38a9'),
 '랭킹': '24',
 '배우 이름': '김성규',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '악인전'],
 '홈페이지': '\nhttps://www.instagram.com/kimsungkyu_official/\n',
 '흥행지수': 11612}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38ae'),
 '랭킹': '29',
 '배우 이름': '김소진',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년'],
 '흥행지수': 10280}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38b0'),
 '랭킹': '31',
 '배우 이름': '이규원',
 '성별': '남',
 '직

In [16]:
documents = collection_cine2.find({}).sort("생년월일", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3901'),
 '랭킹': '112',
 '배우 이름': '이재인',
 '생년월일': '2004-02-06',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['발신제한', '봉오동 전투', '어른도감', '아이 캔 스피크'],
 '흥행지수': 2022}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38fc'),
 '랭킹': '107',
 '배우 이름': '남다름',
 '생년월일': '2002-06-13',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['싱크홀', '제8일의 밤'],
 '흥행지수': 2195}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38b6'),
 '랭킹': '37',
 '배우 이름': '김향기',
 '생년월일': '2000-08-09',
 '성별': '여',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년', '증인'],
 '특기': '애교 부리기, 피아노 치기',
 '홈페이지': '\nhttps://twitter.com/smell2001\n',
 '흥행지수': 8613}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38c2'),
 '랭킹': '49',
 '배우 이름': '성유빈',
 '생년월일': '2000-07-25',
 '성별': '남',
 '직업': 

In [17]:
'''
"특기"라는 key 값이 있는 document 만 조회하기
                  ㄴ $exists
'''
documents = collection_cine2.find({"특기" : {"$exists" : True}})

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3895'),
 '다른 이름': '김태평',
 '랭킹': '4',
 '배우 이름': '현빈',
 '생년월일': '1982-09-25',
 '성별': '남',
 '소속사': 'VAST엔터테인먼트',
 '신장/체중': '184cm, 74kg',
 '원어명': '玄彬',
 '직업': '배우',
 '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'],
 '취미': '수영, 농구, 야구',
 '특기': '스노우보드, 사진',
 '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중',
 '홈페이지': '\nhttp://www.vastenm.com/\n',
 '흥행지수': 21073}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38a3'),
 '랭킹': '18',
 '배우 이름': '손현주',
 '생년월일': '1965-06-24',
 '성별': '남',
 '소속사': '페이스 엔터테인

In [18]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 순으로 정렬하기 (오름차순)
'''
documents = collection_cine2.find({"특기" : {"$exists" : True}}).sort("흥행지수")

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3903'),
 '랭킹': '114',
 '배우 이름': '김성오',
 '생년월일': '1978-09-15',
 '성별': '남',
 '신장/체중': '180cm, 70kg',
 '직업': '배우',
 '출연영화': ['해적: 도깨비 깃발', '킹메이커', '죽지않는 인간들의 밤', '불한당: 나쁜 놈들의 세상', '아저씨'],
 '취미': '낚시, 당구',
 '특기': '스노우보드',
 '흥행지수': 1898}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38ff'),
 '다른 이름': '공지철',
 '랭킹': '110',
 '배우 이름': '공유',
 '생년월일': '1979-07-10',
 '성별': '남',
 '신장/체중': '184cm, 74kg',
 '직업': '배우',
 '출연영화': ['서복', '82년생 김지영', '김종욱 찾기'],
 '취미': '헬스, 농구, 영화',
 '특기': '농구, 노래',
 '학교': '경희대학교 연극영화 - 경희대아트퓨전디자인대학원 퍼포밍아트학',
 '흥행지수': 2104}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38fb'),
 '랭킹': '106',
 '배우 이름': '소지섭',
 '생년월일': '1977-11-04',
 '성별': '남',
 '소속사': '피프티원케이',
 '신장/체중': '182cm, 70kg',
 '직업': '배우',
 '출연영화': ['외계+인 1부'],
 '취미': '각종 운동, 음악 감상, 영화 감상',
 '특기': '수영',
 '학교': '청운대학교 방송연기학 학사',
 '홈페이지'

In [19]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 내림차순으로 정렬하기 - 10 개만
'''
documents = \
  collection_cine2.find({"특기" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3895'),
 '다른 이름': '김태평',
 '랭킹': '4',
 '배우 이름': '현빈',
 '생년월일': '1982-09-25',
 '성별': '남',
 '소속사': 'VAST엔터테인먼트',
 '신장/체중': '184cm, 74kg',
 '원어명': '玄彬',
 '직업': '배우',
 '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'],
 '취미': '수영, 농구, 야구',
 '특기': '스노우보드, 사진',
 '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중',
 '홈페이지': '\nhttp://www.vastenm.com/\n',
 '흥행지수': 21073}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c38a3'),
 '랭킹': '18',
 '배우 이름': '손현주',
 '생년월일': '1965-06-24',
 '성별': '남',
 '소속사': '페이스 엔터테인

In [20]:
'''
생년월일 이라는 key 값이 있는 배우들만
흥행지수 내림차순으로 정렬하기 - 5 개만
'''
documents = \
  collection_cine2.find({"생년월일" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(5)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3893'),
 '랭킹': '2',
 '배우 이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3894'),
 '랭킹': '3',
 '배우 이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}
-----------------------------------------------------------------

In [21]:
'''
생년월일 key 값이 있는 배우들만
흥행지수 순서로 정렬해서 출력하기
내림차순, 5 명만 
배우 이름 key 만 출력하세요
'''

documents = \
  collection_cine2.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수").limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '최무성', '흥행지수': 1402}
--------------------------------------------------------------------------------
{'배우 이름': '송새벽', '흥행지수': 1432}
--------------------------------------------------------------------------------
{'배우 이름': '서예지', '흥행지수': 1451}
--------------------------------------------------------------------------------
{'배우 이름': '오나라', '흥행지수': 1515}
--------------------------------------------------------------------------------
{'배우 이름': '오달수', '흥행지수': 1519}
--------------------------------------------------------------------------------
{'배우 이름': '연우진', '흥행지수': 1528}
--------------------------------------------------------------------------------
{'배우 이름': '우정신', '흥행지수': 1536}
--------------------------------------------------------------------------------
{'배우 이름': '유재명', '흥행지수': 1555}
--------------------------------------------------------------------------------
{'배우 이름': '박병은', '흥행지수': 1598}
-------------------------------------------------------------------------

In [22]:
'''
내림차순 : pymongo.DESCENDING
'''
documents = \
  collection_cine2.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '박해일', '흥행지수': 26750}
--------------------------------------------------------------------------------
{'배우 이름': '박지환', '흥행지수': 24672}
--------------------------------------------------------------------------------
{'배우 이름': '손석구', '흥행지수': 23947}
--------------------------------------------------------------------------------
{'배우 이름': '현빈', '흥행지수': 21073}
--------------------------------------------------------------------------------
{'배우 이름': '최귀화', '흥행지수': 20268}
--------------------------------------------------------------------------------
{'배우 이름': '허동원', '흥행지수': 17930}
--------------------------------------------------------------------------------
{'배우 이름': '변요한', '흥행지수': 17808}
--------------------------------------------------------------------------------
{'배우 이름': '정우성', '흥행지수': 17641}
--------------------------------------------------------------------------------
{'배우 이름': '유해진', '흥행지수': 16882}
-----------------------------------------------------------------

In [23]:
'''
흥행지수가 10000 이상인 배우만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$gte":10000}})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3893'),
 '랭킹': '2',
 '배우 이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
--------------------------------------------------------------------------------
{'_id': ObjectId('63242e2a9fe5962c249c3894'),
 '랭킹': '3',
 '배우 이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}
-----------------------------------------------------------------

In [24]:
'''
흥행지수가 10000 이상인 배우만 조회하기
흥행지수, 배우 이름만 출력하기
흥행지수 내림차순으로 정렬하기
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}}, \
                       {"흥행지수":1,"배우 이름":1, "출연영화":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '박해일',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '흥행지수': 26750}
--------------------------------------------------------------------------------
{'배우 이름': '박지환',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
--------------------------------------------------------------------------------
{'배우 이름': '손석구', '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'], '흥행지수': 23947}
--------------------------------------------------------------------------------
{'배우 이름': '현빈', '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'], '흥행지수': 21073}
--------------------------------------------------------------------------------
{'배우 이름': '최귀화',
 '출연영화': ['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시'],
 '흥행지수': 20268}
--------------------------------------------------------------------------------
{'배우 이름': '허동원',
 '출연영화': ['지푸라기라도 잡고 싶은 짐승들', '특송', '범죄도시2', '악인전'],
 '흥행지수': 17930}
----------

In [25]:
'''
흥행지수가 10000 이상인 배우 중에서
"모가디슈" 라는 영화에 출연한 사람만 조회하기
배우 이름, 흥행지수, 출연영화 만 조회(출력)하기
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":10000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "흥행지수":1, "출연영화":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '송강호',
 '출연영화': ['브로커', '비상선언', '박쥐', '공동경비구역 JSA', '변호인', '택시운전사'],
 '흥행지수': 13137}
--------------------------------------------------------------------------------
{'배우 이름': '김소진',
 '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년'],
 '흥행지수': 10280}
--------------------------------------------------------------------------------


In [26]:
'''
흥행지수가 5000 이 넘으면서
"비상선언" 또는 "이웃사촌" 에 출연한 배우들을 조회하세요
배우 이름, 출연영화 키만 출력하세요
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, "출연영화":"이웃사촌"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

### find() - or

In [27]:
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, \
                        "$or" : [{"출연영화":"비상선언"}, {"출연영화":"이웃사촌"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '송강호',
 '출연영화': ['브로커', '비상선언', '박쥐', '공동경비구역 JSA', '변호인', '택시운전사'],
 '흥행지수': 13137}
--------------------------------------------------------------------------------
{'배우 이름': '김소진',
 '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년'],
 '흥행지수': 10280}
--------------------------------------------------------------------------------
{'배우 이름': '이병헌',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '흥행지수': 7450}
--------------------------------------------------------------------------------
{'배우 이름': '전도연',
 '출연영화': ['비상선언', '지푸라기라도 잡고 싶은 짐승들', '멋진 하루', '밀양'],
 '흥행지수': 6207}
--------------------------------------------------------------------------------


### find() - and

In [28]:
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, \
                        "$and" : [{"출연영화":"비상선언"}, {"출연영화":"그것만이 내 세상"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '이병헌',
 '출연영화': ['비상선언', '번지점프를 하다', '공동경비구역 JSA', '그것만이 내 세상'],
 '흥행지수': 7450}
--------------------------------------------------------------------------------


### find() - nor (not or)

In [29]:
documents = \
  collection_cine2.find({"$nor":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '이규원', '흥행지수': 9486}
--------------------------------------------------------------------------------
{'배우 이름': '허준호', '흥행지수': 9355}
--------------------------------------------------------------------------------
{'배우 이름': '박은빈', '흥행지수': 8977}
--------------------------------------------------------------------------------
{'배우 이름': '진선규', '흥행지수': 8971}
--------------------------------------------------------------------------------
{'배우 이름': '김찬형', '흥행지수': 8862}
--------------------------------------------------------------------------------
{'배우 이름': '정만식', '흥행지수': 8759}
--------------------------------------------------------------------------------
{'배우 이름': '김향기', '흥행지수': 8613}
--------------------------------------------------------------------------------
{'배우 이름': '이광수', '흥행지수': 8476}
--------------------------------------------------------------------------------
{'배우 이름': '서은수', '흥행지수': 8306}
-------------------------------------------------------------------------

In [30]:
documents = \
  collection_cine2.find({"$and":[{"흥행지수": {"$lte":10000}}, {"흥행지수":{"$gte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '이규원', '흥행지수': 9486}
--------------------------------------------------------------------------------
{'배우 이름': '허준호', '흥행지수': 9355}
--------------------------------------------------------------------------------
{'배우 이름': '박은빈', '흥행지수': 8977}
--------------------------------------------------------------------------------
{'배우 이름': '진선규', '흥행지수': 8971}
--------------------------------------------------------------------------------
{'배우 이름': '김찬형', '흥행지수': 8862}
--------------------------------------------------------------------------------
{'배우 이름': '정만식', '흥행지수': 8759}
--------------------------------------------------------------------------------
{'배우 이름': '김향기', '흥행지수': 8613}
--------------------------------------------------------------------------------
{'배우 이름': '이광수', '흥행지수': 8476}
--------------------------------------------------------------------------------
{'배우 이름': '서은수', '흥행지수': 8306}
-------------------------------------------------------------------------

In [31]:
documents = \
  collection_cine2.find({"$or":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

{'배우 이름': '박해일', '흥행지수': 26750}
--------------------------------------------------------------------------------
{'배우 이름': '박지환', '흥행지수': 24672}
--------------------------------------------------------------------------------
{'배우 이름': '손석구', '흥행지수': 23947}
--------------------------------------------------------------------------------
{'배우 이름': '현빈', '흥행지수': 21073}
--------------------------------------------------------------------------------
{'배우 이름': '최귀화', '흥행지수': 20268}
--------------------------------------------------------------------------------
{'배우 이름': '허동원', '흥행지수': 17930}
--------------------------------------------------------------------------------
{'배우 이름': '변요한', '흥행지수': 17808}
--------------------------------------------------------------------------------
{'배우 이름': '정우성', '흥행지수': 17641}
--------------------------------------------------------------------------------
{'배우 이름': '유해진', '흥행지수': 16882}
-----------------------------------------------------------------

### find() - in(포함되어 있음), nin(not in)(포함되어 있지 않음)

In [32]:
'''
흥행지수가 12619, 15263 인 배우만 조회하기 - $in 
배우 이름, 흥행지수 만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$in":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
                                  
for document in documents:
    pprint.pprint(document)    

In [33]:
'''
흥행지수가 12619, 15263 이 아닌 배우만 조회하기 -  $nin
배우 이름, 흥행지수만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$nin":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
for document in documents:
    pprint.pprint(document)    

{'배우 이름': '박해일', '흥행지수': 26750}
{'배우 이름': '박지환', '흥행지수': 24672}
{'배우 이름': '손석구', '흥행지수': 23947}
{'배우 이름': '현빈', '흥행지수': 21073}
{'배우 이름': '최귀화', '흥행지수': 20268}
{'배우 이름': '허동원', '흥행지수': 17930}
{'배우 이름': '변요한', '흥행지수': 17808}
{'배우 이름': '정우성', '흥행지수': 17641}
{'배우 이름': '유해진', '흥행지수': 16882}
{'배우 이름': '하준', '흥행지수': 16518}
{'배우 이름': '김성균', '흥행지수': 15438}
{'배우 이름': '정재광', '흥행지수': 15335}
{'배우 이름': '음문석', '흥행지수': 15266}
{'배우 이름': '안성기', '흥행지수': 15250}
{'배우 이름': '임윤아', '흥행지수': 15222}
{'배우 이름': '이정재', '흥행지수': 15122}
{'배우 이름': '남문철', '흥행지수': 13927}
{'배우 이름': '손현주', '흥행지수': 13418}
{'배우 이름': '송강호', '흥행지수': 13137}
{'배우 이름': '박지영', '흥행지수': 12663}
{'배우 이름': '전혜진', '흥행지수': 12599}
{'배우 이름': '김윤석', '흥행지수': 12484}
{'배우 이름': '고경표', '흥행지수': 12411}
{'배우 이름': '김성규', '흥행지수': 11612}
{'배우 이름': '조우진', '흥행지수': 11591}
{'배우 이름': '이주원', '흥행지수': 11397}
{'배우 이름': '조인성', '흥행지수': 10924}
{'배우 이름': '허성태', '흥행지수': 10292}
{'배우 이름': '김소진', '흥행지수': 10280}
{'배우 이름': '신시아', '흥행지수': 10260}
{'배우 이름': '이규원', '흥행지수': 9486}
{'배우 이름': '

In [34]:
'''
흥행지수가 12619, 15263 이 아니고(아니면서)
10000 이상인 배우 이름과 흥행지수를 조회하세요
  ㄴ 10000 미만이 아닌
[힌트]
  $nor 와 $in 같이 사용하기
  
1)   흥행지수가 12619, 15263 인 조건

     {"흥행지수":{"$in":[12619, 15263]}
     
     
2)   흥행지수가 10000 미만인 조건

     {"흥행지수":{"$lt":10000}
  
3)   {"$nor" : [{"흥행지수":{"$in":[12619, 15263]}, {"흥행지수":{"$lt":10000}]}
'''
documents = \
  collection_cine2.find({"$nor" : [{"흥행지수":{"$in":[12619, 15263]}}, \
                                  {"흥행지수":{"$lt":10000}}]},\
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '박해일', '흥행지수': 26750}
{'배우 이름': '박지환', '흥행지수': 24672}
{'배우 이름': '손석구', '흥행지수': 23947}
{'배우 이름': '현빈', '흥행지수': 21073}
{'배우 이름': '최귀화', '흥행지수': 20268}
{'배우 이름': '허동원', '흥행지수': 17930}
{'배우 이름': '변요한', '흥행지수': 17808}
{'배우 이름': '정우성', '흥행지수': 17641}
{'배우 이름': '유해진', '흥행지수': 16882}
{'배우 이름': '하준', '흥행지수': 16518}
{'배우 이름': '김성균', '흥행지수': 15438}
{'배우 이름': '정재광', '흥행지수': 15335}
{'배우 이름': '음문석', '흥행지수': 15266}
{'배우 이름': '안성기', '흥행지수': 15250}
{'배우 이름': '임윤아', '흥행지수': 15222}
{'배우 이름': '이정재', '흥행지수': 15122}
{'배우 이름': '남문철', '흥행지수': 13927}
{'배우 이름': '손현주', '흥행지수': 13418}
{'배우 이름': '송강호', '흥행지수': 13137}
{'배우 이름': '박지영', '흥행지수': 12663}
{'배우 이름': '전혜진', '흥행지수': 12599}
{'배우 이름': '김윤석', '흥행지수': 12484}
{'배우 이름': '고경표', '흥행지수': 12411}
{'배우 이름': '김성규', '흥행지수': 11612}
{'배우 이름': '조우진', '흥행지수': 11591}
{'배우 이름': '이주원', '흥행지수': 11397}
{'배우 이름': '조인성', '흥행지수': 10924}
{'배우 이름': '허성태', '흥행지수': 10292}
{'배우 이름': '김소진', '흥행지수': 10280}
{'배우 이름': '신시아', '흥행지수': 10260}


### find() - skip() / limit()
- skip(n) - 시작 부분에서 n 개 건너뜀 (n+1 부터 검색함)
- limit(n) - n 개만 조회함

In [35]:
'''
흥행지수가 10000 을 초과하는 배우들의
배우 이름, 흥행지수를 조회하세요
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gt":10000}},\
                       {"흥행지수":1, "배우 이름":1, "출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
print('-' * 80)    
  

{'배우 이름': '박해일',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '흥행지수': 26750}
{'배우 이름': '박지환',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
{'배우 이름': '손석구', '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'], '흥행지수': 23947}
{'배우 이름': '현빈', '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'], '흥행지수': 21073}
{'배우 이름': '최귀화',
 '출연영화': ['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시'],
 '흥행지수': 20268}
{'배우 이름': '허동원',
 '출연영화': ['지푸라기라도 잡고 싶은 짐승들', '특송', '범죄도시2', '악인전'],
 '흥행지수': 17930}
{'배우 이름': '변요한',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아'],
 '흥행지수': 17808}
{'배우 이름': '정우성',
 '출연영화': ['헌트', '지푸라기라도 잡고 싶은 짐승들', '태양은 없다', '강철비2: 정상회담', '호우시절', '증인'],
 '흥행지수': 17641}
{'배우 이름': '유해진',
 '출연영화': ['공조2: 인터내셔날', '타짜', '승리호', '봉오동 전투', '택시운전사', '말모이'],
 '흥행지수': 16882}
{'배우 이름': '하준', '출연영화': ['잔칫날', '범죄도시2'], '흥행지수': 16518}
{'배우 이름': '김성균',
 '출연영화': ['싱크홀', '한산: 용의 출현', 

In [36]:
'''
모가디슈에 출연한 배우들의 배우 이름, 출연영화를 출력하세요
'''
documents = \
  collection_cine2.find({"출연영화":"모가디슈"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

{'배우 이름': '김윤석',
 '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우 이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우 이름': '김소진', '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우 이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}
{'배우 이름': '정만식',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}
{'배우 이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우 이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우 이름': '박경혜', '출연영화': ['모가디슈', '리미트', '메기', '1987']}


In [37]:
'''
모가디슈나 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents = \
  collection_cine2.find({"$or":[{"출연영화":"모가디슈"},{"출연영화":"타짜"}]},\
                        {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '유해진', '출연영화': ['공조2: 인터내셔날', '타짜', '승리호', '봉오동 전투', '택시운전사', '말모이']}
{'배우 이름': '김윤석',
 '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우 이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우 이름': '김소진', '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우 이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}
{'배우 이름': '정만식',
 '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}
{'배우 이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우 이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우 이름': '박경혜', '출연영화': ['모가디슈', '리미트', '메기', '1987']}


In [38]:
'''
모가디슈와 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents =\
  collection_cine2.find({"출연영화":{"$all":["모가다슈", "타짜"]}},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

In [39]:
documents = collection_cine2.find({},{"_id":0})
# print(len(list(documents)))
for document in documents:
    pprint.pprint(document)

{'랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
{'랭킹': '2',
 '배우 이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
{'랭킹': '3',
 '배우 이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}
{'다른 이름': '김태평',
 '랭킹': '4',
 '배우 이름': '현빈',
 '생년월일': '1982-09-25',
 '성별': '남',
 '소속사': 'VAST엔터테인먼트',
 '신장/체중': '184cm, 74kg',
 '원어명': '玄彬',
 '직업': '배우',
 '출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조'],
 '취미': '수영, 농구, 야구',
 '특기': '스노우보드, 사진',
 '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중',
 '홈페이지': '\nhttp://www.vastenm.com/\n',
 '흥행지수': 21073}
{'랭킹': '5',
 '배우 이름':

In [40]:
'''
5 편의 영화에 출연한 배우만 검색하기 : $size
배우 이름, 출연영화
'''

documents = \
  collection_cine2.find({"출연영화":{"$size":5}}, {"배우 이름":1,"출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    

{'배우 이름': '변요한',
 '출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아']}
{'배우 이름': '김성균',
 '출연영화': ['싱크홀', '한산: 용의 출현', '화이 : 괴물을 삼킨 아이', '군도 : 민란의 시대', '프리즌']}
{'배우 이름': '정재광', '출연영화': ['범죄도시2', '파이프라인', '낫아웃', '버티고', '여름밤']}
{'배우 이름': '남문철', '출연영화': ['범죄도시2', '애비규환', '독전', '4등', '유열의 음악앨범']}
{'배우 이름': '전혜진', '출연영화': ['헌트', '불한당: 나쁜 놈들의 세상', '시인의 사랑', '뺑반', '택시운전사']}
{'배우 이름': '김소진', '출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우 이름': '강하늘', '출연영화': ['해적: 도깨비 깃발', '비와 당신의 이야기', '해피 뉴 이어', '좋아해줘', '동주']}
{'배우 이름': '이정현', '출연영화': ['헤어질 결심', '죽지않는 인간들의 밤', '반도', '리미트', '명량']}
{'배우 이름': '진구',
 '출연영화': ['마녀(魔女) Part2. The Other One', '내겐 너무 소중한 너', '기담', '연평해전', '명량']}
{'배우 이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우 이름': '고창석',
 '출연영화': ['싱크홀', '니 부모 얼굴이 보고 싶다', '서양골동양과자점 앤티크', '택시운전사', '1987']}
{'배우 이름': '김강우', '출연영화': ['내일의 기억', '귀문', '돈의 맛', '새해전야', '하하하']}
{'배우 이름': '배유람',
 '출연영화': ['내일의 기억', '파이프라인', '찬실이는 복도 많지', '연애 빠진 로맨스', '소셜포

### elemMatch
- 각 document 의 key의 값으로 저장되어 있는 list에
- 적어도 한 개 이상의 요소(element : item)가 여러 개의 조건을 
- 동시에 만족하는 경우
- 조건인 하나인 경우에는 사용하지 않음

In [41]:
documents = collection_cine2.find({}).limit(3)
for document in documents:
    pprint.pprint(document)

{'_id': ObjectId('63242e2a9fe5962c249c3892'),
 '랭킹': '1',
 '배우 이름': '박해일',
 '생년월일': '1977-01-26',
 '성별': '남',
 '소속사': '휴메인엔터테인먼트',
 '신장/체중': '177cm, 65kg',
 '직업': '배우',
 '출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유'],
 '취미': '영화감상, 음악감상, 컴퓨터게임',
 '특기': '노래, 기타연주, 탁구, 야구',
 '학교': '남서울대학교 영어학 (중퇴)',
 '흥행지수': 26750}
{'_id': ObjectId('63242e2a9fe5962c249c3893'),
 '랭킹': '2',
 '배우 이름': '박지환',
 '생년월일': '1980-09-05',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['유체이탈자',
          '한산: 용의 출현',
          '해적: 도깨비 깃발',
          '지푸라기라도 잡고 싶은 짐승들',
          '범죄도시2',
          '봉오동 전투'],
 '흥행지수': 24672}
{'_id': ObjectId('63242e2a9fe5962c249c3894'),
 '랭킹': '3',
 '배우 이름': '손석구',
 '생년월일': '1983-00-00',
 '성별': '남',
 '직업': '배우',
 '출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반'],
 '흥행지수': 23947}


In [42]:
'''
직업이 가수인 배우 중에서 
흥행지수가 가장 높은 순서로 (내림차순) 정렬해서
배우 이름, 직업, 흥행 지수를 조회하세요
'''
documents = collection_cine2.find({"직업":"가수"},\
                                 {"배우 이름":1, "직업":1, "흥행지수":1, "_id":0})\
                           .sort("흥행지수", pymongo.DESCENDING) 
for document in documents:
    pprint.pprint(document)

{'배우 이름': '임윤아', '직업': '가수', '흥행지수': 15222}
{'배우 이름': '택연', '직업': '가수', '흥행지수': 6635}
{'배우 이름': '이지은', '직업': '가수', '흥행지수': 1851}
{'배우 이름': '오세훈', '직업': '가수', '흥행지수': 1690}


In [43]:
'''
collection_elementmatch 컬렉션 생성하기
'''
cine.collection_elementmatch
# cine["collection_elementmatch"]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cine'), 'collection_elementmatch')

In [44]:
elematch = cine.collection_elementmatch

In [45]:
doc1 = {"results" : [81, 85, 89]}
doc2 = {"results" : [81, 85, 89, 72]}
doc3 = {"results" : [81, 85, 89, 97]}
doc4 = {"results" : [81, 85, 89, 72, 97]}
doc5 = {"results" : [77, 86, 93]}

elematch.insert_many(
    [
        doc1, doc2, doc3, doc4, doc5
    ]
)

In [46]:
'''
"$gte":90  <- 90 점 이상인 data 가 있는 document 조회(검색)하기
"$lt":80   <- 80 점 미만인 data 가 있는 document 조회하기
'''
documents = elematch.find({"results":{"$gte":90, "$lt":80}}, {"_id":0})
for document in documents:
    pprint.pprint(document)

{'results': [81, 85, 89, 72, 97]}
{'results': [77, 86, 93]}


In [47]:
'''
각 조건별로 하나라도 만족하는 element가 있는 경우
'''
documents = elematch.find({"results":{"$elemMatch":{"$gte":85, "$lt":90}}}, {"_id":0})
for document in documents:
    print(document)

{'results': [81, 85, 89]}
{'results': [81, 85, 89, 72]}
{'results': [81, 85, 89, 97]}
{'results': [81, 85, 89, 72, 97]}
{'results': [77, 86, 93]}


In [48]:
doc6 = {"results": [82, 85, 88]}
doc7 = {"results": [92, 95, 98]}

elematch.insert_many(
    [
        doc6, doc7        
    ]
)

In [49]:
for document in elematch.find({"results" : {"$elemMatch":{"$gte":80, "$lt":85}}}, {"_id":0}):
    print(document)

{'results': [81, 85, 89]}
{'results': [81, 85, 89, 72]}
{'results': [81, 85, 89, 97]}
{'results': [81, 85, 89, 72, 97]}
{'results': [82, 85, 88]}


In [50]:
'''
collection_cine2 에서
출연영화 key 만 10 개 조회하기
'''
for document in collection_cine2.find({}, {"출연영화":1, "_id":0}).limit(10):
    print(document)

{'출연영화': ['한산: 용의 출현', '헤어질 결심', '와이키키 브라더스', '후아유']}
{'출연영화': ['유체이탈자', '한산: 용의 출현', '해적: 도깨비 깃발', '지푸라기라도 잡고 싶은 짐승들', '범죄도시2', '봉오동 전투']}
{'출연영화': ['범죄도시2', '연애 빠진 로맨스', '뺑반']}
{'출연영화': ['공조2: 인터내셔날', '만추', '협상', '공조']}
{'출연영화': ['부기나이트', '기방도령', '범죄도시2', '원더풀 고스트', '택시운전사', '조작된 도시']}
{'출연영화': ['지푸라기라도 잡고 싶은 짐승들', '특송', '범죄도시2', '악인전']}
{'출연영화': ['한산: 용의 출현', '자산어보', '태양은 움직이지 않는다', '당신, 거기 있어줄래요', '소셜포비아']}
{'출연영화': ['헌트', '지푸라기라도 잡고 싶은 짐승들', '태양은 없다', '강철비2: 정상회담', '호우시절', '증인']}
{'출연영화': ['공조2: 인터내셔날', '타짜', '승리호', '봉오동 전투', '택시운전사', '말모이']}
{'출연영화': ['잔칫날', '범죄도시2']}


In [51]:
'''
출연영화에 '도굴' 이라는 영화가 있는 document 만 조회하기
출연영화 key 만 출력함
'''
for document in collection_cine2.find({"출연영화":"도굴"}, {"출연영화":1, "_id":0}):
    print(document)

{'출연영화': ['외계+인 1부', '발신제한', '도굴', '킹메이커', '서복', '국가부도의 날']}


In [52]:
'''
출연영화에 '모가디슈' 이라는 영화가 포함되어 있는 document 만 조회하기
출연영화 key 만 출력함
'''
for document in collection_cine2.find({"출연영화":"모가디슈"}, {"출연영화":1, "_id":0}):
    print(document)

{'출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'출연영화': ['모가디슈', '클래식']}
{'출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년']}
{'출연영화': ['모가디슈', '국가부도의 날']}
{'출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}
{'출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'출연영화': ['모가디슈', '리미트', '메기', '1987']}


In [53]:
'''
출연영화가 "모가디슈", "클래식" 두 개인 document 조회하기
'''
for document in collection_cine2.find({"출연영화":["모가디슈", "클래식"]}, {"출연영화":1, "_id":0}):
    print(document)

{'출연영화': ['모가디슈', '클래식']}


In [54]:
'''
출연영화가 "모가디슈", "메기" 두 개인 document 조회하기
'''
for document in collection_cine2.find({"출연영화":["모가디슈", "메기"]}, {"출연영화":1, "_id":0}):
    print(document)

In [55]:
'''
출연영화에 "모가디슈", "메기" 가 포함되어 있는 document
'''
documents = \
  collection_cine2.find({"출연영화":{"$all":["모가디슈", "메기"]}}, \
                        {"출연영화":1, "_id":0})

for document in documents:
    print(document)

{'출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'출연영화': ['모가디슈', '리미트', '메기', '1987']}


In [56]:
'''
출연영화에 "국가부도의 날" 이 포함되어 있는 document
'''
for document in collection_cine2.find({"출연영화":"국가부도의 날"}, {"출연영화":1,"_id":0}):
    print(document)


{'출연영화': ['외계+인 1부', '발신제한', '도굴', '킹메이커', '서복', '국가부도의 날']}
{'출연영화': ['모가디슈', '국가부도의 날']}
{'출연영화': ['싱크홀', '니 부모 얼굴이 보고 싶다', '괴기맨숀', '배심원들', '국가부도의 날', '말모이']}


In [57]:
'''
"국가부도의 날" 에 출연한 배우들 중에서 
"흥행지수" 가 높은 순으로 
배우 이름, 직업, 흥행지수를 조회하세요
'''
documents = collection_cine2.find({"출연영화":"국가부도의 날"},\
                                  {"배우 이름":1, "직업":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING)
for document in documents:
    pprint.pprint(document)

{'배우 이름': '조우진', '직업': '배우', '흥행지수': 11591}
{'배우 이름': '허준호', '직업': '배우', '흥행지수': 9355}
{'배우 이름': '김홍파', '직업': '배우', '흥행지수': 4793}
